## Heart Failure Classification using PandasAI

El dataset está sacado de kaggle en este enlace, <a href="https://www.kaggle.com/datasets/aadarshvelu/heart-failure-prediction-clinical-records/data">Heart Failure Prediction - Clinical Records 🏥 </a>

Este dataset tiene datos de los pacientes y nos indica si estos sufriesen un ataque al corazón si sobrevivirían o no a una operación así como al translado urgente al hospital

In [46]:
import pandas as pd
import pandasai
import os 
from dotenv import dotenv_values
import datetime

In [47]:
# Environment key
keys = dotenv_values("../.env-token")
os.environ["PANDASAI_API_KEY"] = keys["PANDASAI_API_KEY"]

In [48]:
from pandasai import SmartDataframe
from pandasai.llm import OpenAI

In [49]:
init_time = datetime
file_dir="Data/heart_failure_clinical_records.csv"
file = pd.read_csv(file_dir)
file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       5000 non-null   float64
 1   anaemia                   5000 non-null   int64  
 2   creatinine_phosphokinase  5000 non-null   int64  
 3   diabetes                  5000 non-null   int64  
 4   ejection_fraction         5000 non-null   int64  
 5   high_blood_pressure       5000 non-null   int64  
 6   platelets                 5000 non-null   float64
 7   serum_creatinine          5000 non-null   float64
 8   serum_sodium              5000 non-null   int64  
 9   sex                       5000 non-null   int64  
 10  smoking                   5000 non-null   int64  
 11  time                      5000 non-null   int64  
 12  DEATH_EVENT               5000 non-null   int64  
dtypes: float64(3), int64(10)
memory usage: 507.9 KB


Como descripción simplemente vamos a decirle el número de columnas y cuál es la que tenemos como 'target' para la clasificación.

In [50]:
# Aquí pondemos una pequeña descripción del dataset
DESCRIPTION = """
The dataset has 13 columns, the target columns is DEATH_EVENT with 2 values
"""

Vamos a pasarle como configuración que use el modelo gpt-4-turbo, al igual que AIDE para tenerlos en igualdad de condiciones en lo que al LLM respecta ya que este no realiza varias iteraciones sobre varios programas buscando cuál es el más óptimo. Este sacará un código, lo ejecutará y nosotros sacaremos el código. 

Todas las consultas e input+output prompts los podemos encontrar en el fichero pandasai.log. No es recomendable entrar a buscar una consulta simple ya que cada vez que hacemos una llamada a PandasAI este copia input y output en el fichero pandasai.log por lo que es un fichero de tamaño considerable con consultas no parecidas sobre las que pandasai tiene constancia para aumentar la velocidad en las siguientes consultas si estas son monotema.

In [51]:

llm = OpenAI(model="gpt-4o") # Este es el modelo de OpenAI que vamos a utilizar
smartDataFrame = SmartDataframe(file, description=DESCRIPTION, config={"llm":llm})

Ahora hacemos el chat y que nos proporcione una solución

In [52]:
response = smartDataFrame.chat("Make a classifier for this dataset that predicts if a patient will survive a heart failure? (DEATH_EVENT=1 --> True class)")
print(response)

Accuracy: 0.993

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       698
           1       0.99      0.99      0.99       302

    accuracy                           0.99      1000
   macro avg       0.99      0.99      0.99      1000
weighted avg       0.99      0.99      0.99      1000


Confusion Matrix:
[[694   4]
 [  3 299]]



Coste de ejecuciónn --> <label style="color:red">Parece que no me ha costado nada</label> < de $.01

Sacamos el código y lo ejecutamos

In [53]:
print(smartDataFrame.last_code_executed)

df = dfs[0]
X = df.drop('DEATH_EVENT', axis=1)
y = df['DEATH_EVENT']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
result = {'type': 'string', 'value': f"""Accuracy: {accuracy}

Classification Report:
{report}

Confusion Matrix:
{conf_matrix}"""}


Como podemos ver comparando el código los código de las celdas pegadas a esta no coinciden del todo ya que PandasAI, a diferencia de otras, no devuelve los imports ni printea el resultado ya que este lo devuelve en forma de un report

In [60]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
df = file
X = df.drop('DEATH_EVENT', axis=1)
y = df['DEATH_EVENT']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
result = {'type': 'string', 'value': f"""Accuracy: {accuracy}

Classification Report:
{report}

Confusion Matrix:
{conf_matrix}"""}

print(accuracy)
print(report)
print(conf_matrix)

0.992
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       698
           1       0.98      0.99      0.99       302

    accuracy                           0.99      1000
   macro avg       0.99      0.99      0.99      1000
weighted avg       0.99      0.99      0.99      1000

[[692   6]
 [  2 300]]


Observando el reporte de la clasificación tenemos puntuaciones de accuracy, recall, f1-score y una matriz de confusió casi perfecta por lo que PandasAI ha sido capaz de clasificar perfectamente el dataset.